In [ ]:
# must download when new kernel created
%pip install git+https://github.com/huggingface/transformers -qqq
%pip install bitsandbytes -qqq
%pip install huggingface_hub -qqq
%pip install peft -qqq
%pip install accelerate -qqq
%pip install trl -qqq
%pip install wandb -qqq
%pip install ipywidgets -qqq

%pip install torch==2.1.0 torchvision torchaudio 

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ongjjbrandon/mistral-pjf-v3/model-gaflufph:v0', type='model')
artifact_dir = artifact.download()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login
# base_model, adapter, new_model = "mistralai/Mistral-7B-v0.1" , "./artifacts/model-gaflufph:v0/", "ogbrandt/bfloat-16-merge"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cpu")
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()
model.save_pretrained("/home/ubuntu/remote-training/merged_models")

In [1]:
!nvidia-smi

Wed Jan 31 03:24:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   23C    P0              53W / 300W |      0MiB / 23028MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# better merge?
# Clear the memory footprint
import torch

# del model
# torch.cuda.empty_cache()

base_model = "ogbrandt/bfloat-16-merge"
adapter = "ogbrandt/mistral-7b-non-instruction-pjf-ft-gpt35-dpo-v0"

# Reload the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})

model = PeftModel.from_pretrained(base_model_reload, adapter)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

In [4]:
# from huggingface_hub import login
# login()

model.push_to_hub("mist7b-pjf-ft-dpo", use_temp_dir=True)
tokenizer.push_to_hub("mist7b-pjf-ft-dpo", use_temp_dir=True)

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ogbrandt/mist7b-pjf-ft-dpo/commit/826e50d0e6bc05920dfe4ea8af51f1c3e6cf1a85', commit_message='Upload tokenizer', commit_description='', oid='826e50d0e6bc05920dfe4ea8af51f1c3e6cf1a85', pr_url=None, pr_revision=None, pr_num=None)